In [1]:
import os
import json
import ast
import numpy as np
import pandas as pd
from collections import Counter
pd.set_option('display.max_colwidth', 900)

# Create mapping between cards and integers

In [28]:
def get_cardset(drafts):
    cardset = set()
    for draft in drafts:
        for pack in draft:
            for pick in pack:
                for card in pick:
                    cardset.add(card)
    carddict = {card: i for i, card in enumerate(cardset)}
    return carddict

In [30]:
cardset = get_cardset(drafts)
invcardset = {v: k for k, v in cardset.items()}

# Preprocessing

In [29]:
def parse_draft_to_standard_format(flooey_draft):
    draft = []
    for pack in flooey_draft['packs']:
        pack_list = []
        for pick in pack['picks']:
            cards = pick['cards']
            cards = sorted(cards, key=lambda x: x!=pick['pick'])
            pack_list.append(cards)
        draft.append(pack_list)
    return draft

In [23]:
draftsjson = json.load(open('allgrn.json'))
drafts = [parse_draft_to_standard_format(draft['Draft']) for draft in draftsjson]

In [37]:
len(drafts)

2130

In [24]:
drafts[0]

[[[u'Risk Factor',
   u'Never Happened',
   u'Hired Poisoner',
   u'Dimir Informant',
   u'Garrison Sergeant',
   u'Take Heart',
   u'Skyknight Legionnaire',
   u'Pitiless Gorgon',
   u'Disdainful Stroke',
   u'Golgari Locket',
   u'Severed Strands',
   u'Gird for Battle',
   u'Sunhome Stalwart',
   u'Might of the Masses',
   u'Selesnya Guildgate'],
  [u'Dimir Spybug',
   u'Wild Ceratok',
   u'Wishcoin Crab',
   u'Ornery Goblin',
   u'Skyline Scout',
   u'Leapfrog',
   u'Pitiless Gorgon',
   u'Hypothesizzle',
   u'Fearless Halberdier',
   u'Veiled Shade',
   u'Muse Drake',
   u'Thought Erasure',
   u'Gatekeeper Gargoyle',
   u'Selesnya Guildgate'],
  [u'House Guildmage',
   u'Crushing Canopy',
   u'Capture Sphere',
   u'Darkblade Agent',
   u'Vicious Rumors',
   u'Douser of Lights',
   u'Hitchclaw Recluse',
   u'Torch Courier',
   u'Sumala Woodshaper',
   u'Wall of Mist',
   u'Garrison Sergeant',
   u'Worldsoul Colossus',
   u'Izzet Guildgate'],
  [u'Whisper Agent',
   u'Blade Instruct

# Convert drafts to vectors

In [34]:
def vectorize_drafts(drafts):
    cardset = get_cardset(drafts)
    flooey_drafts = [parse_draft_to_flooey_format(draft) for draft in drafts]
    return [vectorize_draft(draft, cardset) for draft in flooey_drafts]

def vectorize_draft(draft, cardset):
    draft_x = np.zeros([3, len(draft[0][0]), len(cardset) * 2])
    draft_y = np.zeros([3, len(draft[0][0]), len(cardset)])
    pile = np.zeros(len(cardset))
    for pa, pack in enumerate(draft):
        for pi, pick in enumerate(pack):
            draft_y[pa][pi][cardset.get(pick[0], -1)] = 1
            x1 = np.zeros(len(cardset))
            for card in pick:
                if card in cardset:
                    x1[cardset[card]] += 1
                else:
                    x1[-1] += 1
            draft_x[pa][pi] = np.append(x1, pile)
            pile[cardset.get(pick[0], -1)] += 1
            
    return {'draft_x': draft_x, 'draft_y': draft_y}

# Rudimentary train/test split

In [70]:
import random
random.shuffle(drafts)
split_no = int(len(drafts) * 0.8)
train = drafts[:split_no]
test = drafts[split_no:]
print(len(train), len(test))

(1704, 426)


# Modeling

In [77]:
def DraftGenerator(trainset, batch_size=20):
    while True:
        for i in range(0, len(trainset) / batch_size):
            flooey_drafts = trainset[i*batch_size:(i+1)*batch_size]
            draftsv = [vectorize_draft(draft, cardset) for draft in flooey_drafts]
            X, Y = [], []
            for draft in draftsv:
                draft_x = draft['draft_x']
                draft_y = draft['draft_y']
                for pack in range(len(draft_x)):
                    for pick in range(len(draft_x[pack])):
                        Y.append(draft_y[pack][pick])
                        X.append(draft_x[pack][pick].flatten())
            yield (np.array(X), np.array(Y))

In [92]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [114]:
model = Sequential()
model.add(Dense(
    512, activation='relu',
    input_dim=len(cardset) * 2
))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Dense(len(cardset), activation='softmax'))
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [115]:
dg = DraftGenerator(train)

In [ ]:
batch_size = 20
model.fit_generator(dg, steps_per_epoch=len(train) / batch_size, epochs=10)

In [218]:
dgtest = DraftGenerator(test)

In [225]:
model.evaluate_generator(dgtest, steps=len(test) / batch_size)[1]

0.5889947073800224

# Show picks from some sample drafts

In [154]:
testv = [vectorize_draft(draft, cardset) for draft in test]
X, Y = [], []
for draft in testv:
    draft_x = draft['draft_x']
    draft_y = draft['draft_y']
    for pack in range(len(draft_x)):
        for pick in range(len(draft_x[pack])):
            Y.append(draft_y[pack][pick])
            X.append(draft_x[pack][pick].flatten())
test_y = np.array(Y)
test_x = np.array(X)

In [184]:
ypred = model.predict(test_x)
ypred = [invcardset[np.argmax(yp)] for yp in ypred]
ytrue = []
cards_list = []
for draft in test:
    for pack in draft:
        for pick in pack:
            ytrue.append(pick[0])
            cards_list.append(pick)

In [189]:
df = pd.DataFrame({
    'options': cards_list[:19000],
    'predicted': ypred[:19000],
    'taken': ytrue[:19000]
})

In [209]:
df.iloc[:45]

,options,predicted,taken
0,"[Affectionate Indrik, Ledev Guardian, Sumala Woodshaper, Leapfrog, Hammer Dropper, Rhizome Lurcher, Siege Wurm, Devious Cover-Up, Parhelion Patrol, Vicious Rumors, Skyknight Legionnaire, Inspiring Unicorn, Molderhulk, Firemind's Research, Izzet Guildgate]",Skyknight Legionnaire,Affectionate Indrik
1,"[Necrotic Wound, Pack's Favor, Unexplained Disappearance, Boros Locket, Rhizome Lurcher, Luminous Bonds, Goblin Electromancer, Crushing Canopy, Capture Sphere, Darkblade Agent, Vicious Rumors, Lotleth Giant, Crush Contraband, Izzet Guildgate]",Luminous Bonds,Necrotic Wound
2,"[Murmuring Mystic, Vicious Rumors, Spinal Centipede, Golgari Locket, Leapfrog, Radical Idea, Hammer Dropper, Dead Weight, Sumala Woodshaper, Maximize Altitude, Gravitic Punch, Smelt-Ward Minotaur, Selesnya Guildgate]",Murmuring Mystic,Murmuring Mystic
3,"[Dimir Guildgate, Cosmotronic Wave, Disdainful Stroke, Radical Idea, Maniacal Rage, Intrusive Packbeast, Douser of Lights, Hitchclaw Recluse, Loxodon Restorer, Swathcutter Giant, Flower/Flourish, Golgari Raiders]",Radical Idea,Dimir Guildgate
4,"[Beacon Bolt, Severed Strands, Fearless Halberdier, Whisper Agent, Torch Courier, Collar the Culprit, Vedalken Mesmerist, Crushing Canopy, Molderhulk, Guild Summit, Golgari Guildgate]",Whisper Agent,Beacon Bolt
5,"[Gatekeeper Gargoyle, Undercity Uprising, Devkarin Dissident, Goblin Locksmith, Vicious Rumors, Boros Locket, Wojek Bodyguard, Maximize Altitude, Selective Snare, Boros Guildgate]",Wojek Bodyguard,Gatekeeper Gargoyle
6,"[Gateway Plaza, Generous Stray, Douser of Lights, Take Heart, Dazzling Lights, Dimir Locket, Inspiring Unicorn, Invert/Invent, Izzet Guildgate]",Invert/Invent,Gateway Plaza
7,"[Devious Cover-Up, Garrison Sergeant, Fearless Halberdier, Mephitic Vapors, Izzet Locket, Pack's Favor, Crush Contraband, Selesnya Guildgate]",Devious Cover-Up,Devious Cover-Up
8,"[Devious Cover-Up, Ledev Guardian, Sumala Woodshaper, Hammer Dropper, Vicious Rumors, Molderhulk, Izzet Guildgate]",Izzet Guildgate,Devious Cover-Up
9,"[Izzet Guildgate, Boros Locket, Crushing Canopy, Vicious Rumors, Lotleth Giant, Crush Contraband]",Izzet Guildgate,Izzet Guildgate


In [210]:
df.iloc[45:90]

,options,predicted,taken
45,"[Chemister's Insight, Fearless Halberdier, Skyknight Legionnaire, Urban Utopia, Spinal Centipede, Mephitic Vapors, Dimir Locket, Golgari Locket, Candlelight Vigil, Hypothesizzle, Muse Drake, Invert/Invent, Justice Strike, Deafening Clarion, Golgari Guildgate]",Deafening Clarion,Chemister's Insight
46,"[Underrealm Lich, Never Happened, Gravitic Punch, Rhizome Lurcher, Barging Sergeant, Vedalken Mesmerist, Crushing Canopy, Ledev Guardian, Loxodon Restorer, Maniacal Rage, Ornery Goblin, Flower/Flourish, Truefire Captain, Selesnya Guildgate]",Underrealm Lich,Underrealm Lich
47,"[Hatchery Spider, Hunted Witness, Sworn Companions, Hired Poisoner, Rubblebelt Boar, Passwall Adept, Dazzling Lights, Selesnya Locket, Hitchclaw Recluse, Fearless Halberdier, Might of the Masses, Book Devourer, Dimir Guildgate]",Hatchery Spider,Hatchery Spider
48,"[Status/Statue, Leapfrog, Pitiless Gorgon, Barrier of Bones, Erstwhile Trooper, Dimir Informant, Dead Weight, Goblin Electromancer, Sure Strike, Justice Strike, Discovery/Dispersal, Selesnya Guildgate]",Status/Statue,Status/Statue
49,"[Whisper Agent, Wishcoin Crab, Wall of Mist, Veiled Shade, Dimir Informant, Ledev Guardian, Selesnya Locket, Prey Upon, Inescapable Blaze, Wand of Vertebrae, Selesnya Guildgate]",Whisper Agent,Whisper Agent
50,"[Darkblade Agent, Collar the Culprit, Wild Ceratok, Intrusive Packbeast, Barrier of Bones, Crushing Canopy, Passwall Adept, Prey Upon, Wand of Vertebrae, Boros Guildgate]",Prey Upon,Darkblade Agent
51,"[Golgari Guildgate, Pack's Favor, Torch Courier, Undercity Uprising, Child of Night, Veiled Shade, Leapfrog, Undercity Necrolisk, Golgari Raiders]",Golgari Guildgate,Golgari Guildgate
52,"[Gateway Plaza, Wary Okapi, Vicious Rumors, Piston-Fist Cyclops, Undercity Uprising, Garrison Sergeant, Generous Stray, Izzet Guildgate]",Piston-Fist Cyclops,Gateway Plaza
53,"[Mephitic Vapors, Fearless Halberdier, Urban Utopia, Dimir Locket, Golgari Locket, Candlelight Vigil, Invert/Invent]",Mephitic Vapors,Mephitic Vapors
54,"[Crushing Canopy, Never Happened, Gravitic Punch, Vedalken Mesmerist, Ledev Guardian, Loxodon Restorer]",Crushing Canopy,Crushing Canopy


In [211]:
df.iloc[90:135]

,options,predicted,taken
90,"[Experimental Frenzy, Notion Rain, Healer's Hawk, Sumala Woodshaper, Hypothesizzle, Vernadi Shieldmate, Pitiless Gorgon, Moodmark Painter, Rhizome Lurcher, Capture Sphere, Hammer Dropper, Conclave Cavalier, Gird for Battle, Lotleth Giant, Dimir Guildgate]",Experimental Frenzy,Experimental Frenzy
91,"[Niv-Mizzet, Parun, Intrusive Packbeast, Capture Sphere, Sworn Companions, Undercity Uprising, Ledev Guardian, Prey Upon, Severed Strands, Selesnya Locket, Barging Sergeant, Never Happened, Inescapable Blaze, Grappling Sundew, Golgari Guildgate]","Niv-Mizzet, Parun","Niv-Mizzet, Parun"
92,"[Sinister Sabotage, Fearless Halberdier, Intrusive Packbeast, Skyknight Legionnaire, Whisper Agent, Wojek Bodyguard, Child of Night, Wishcoin Crab, Deadly Visit, Pack's Favor, Garrison Sergeant, Hellkite Whelp, Selesnya Guildgate]",Sinister Sabotage,Sinister Sabotage
93,"[Unexplained Disappearance, Sworn Companions, Rubblebelt Boar, Hitchclaw Recluse, Cosmotronic Wave, Dimir Locket, Rosemane Centaur, Vicious Rumors, Capture Sphere, Gird for Battle, Flight of Equenauts, Golgari Guildgate]",Capture Sphere,Unexplained Disappearance
94,"[Dimir Informant, Artful Takedown, Capture Sphere, Burglar Rat, Gravitic Punch, Fresh-Faced Recruit, Selesnya Locket, Radical Idea, Notion Rain, Ledev Champion, Boros Guildgate]",Artful Takedown,Dimir Informant
95,"[Izzet Guildgate, Maximize Velocity, Hitchclaw Recluse, Moodmark Painter, Ledev Guardian, Golgari Locket, Gravitic Punch, Centaur Peacemaker, Wall of Mist, Crush Contraband]",Izzet Guildgate,Izzet Guildgate
96,"[Hypothesizzle, Child of Night, Pause for Reflection, Rosemane Centaur, Golgari Locket, Goblin Locksmith, Intrusive Packbeast, Silent Dart, Izzet Guildgate]",Hypothesizzle,Hypothesizzle
97,"[Notion Rain, Hunted Witness, Centaur Peacemaker, Boros Locket, Rubblebelt Boar, Golgari Locket, Intrusive Packbeast, Grappling Sundew]",Notion Rain,Notion Rain
98,"[Hypothesizzle, Notion Rain, Moodmark Painter, Hammer Dropper, Gird for Battle, Lotleth Giant, Dimir Guildgate]",Hypothesizzle,Hypothesizzle
99,"[Barging Sergeant, Ledev Guardian, Selesnya Locket, Never Happened, Grappling Sundew, Golgari Guildgate]",Barging Sergeant,Barging Sergeant


In [212]:
df.iloc[135:180]

,options,predicted,taken
135,"[Beacon Bolt, Generous Stray, Rubblebelt Boar, Never Happened, Boros Locket, Rosemane Centaur, Dimir Informant, Cosmotronic Wave, Sumala Woodshaper, Vedalken Mesmerist, Devkarin Dissident, Street Riot, Nightveil Predator, Camaraderie, Boros Guildgate]",Nightveil Predator,Beacon Bolt
136,"[Roc Charger, Burglar Rat, Barrier of Bones, Vedalken Mesmerist, Centaur Peacemaker, Severed Strands, Vernadi Shieldmate, Pack's Favor, Crushing Canopy, Command the Storm, Urban Utopia, Inescapable Blaze, Undercity Necrolisk, Dimir Guildgate]",Inescapable Blaze,Roc Charger
137,"[Deafening Clarion, Urban Utopia, Torch Courier, Pause for Reflection, Bartizan Bats, Centaur Peacemaker, Passwall Adept, Capture Sphere, Tenth District Guard, Garrison Sergeant, Hellkite Whelp, Arboretum Elemental, Boros Guildgate]",Deafening Clarion,Deafening Clarion
138,"[Firemind's Research, Spinal Centipede, Maniacal Rage, Golgari Locket, Dazzling Lights, Rosemane Centaur, Veiled Shade, Wild Ceratok, Never Happened, Crush Contraband, Molderhulk, Golgari Guildgate]",Rosemane Centaur,Firemind's Research
139,"[Radical Idea, Wojek Bodyguard, Rosemane Centaur, Golgari Locket, Burglar Rat, Crushing Canopy, Unexplained Disappearance, Skyknight Legionnaire, Piston-Fist Cyclops, Molderhulk, Selesnya Guildgate]",Skyknight Legionnaire,Radical Idea
140,"[Dimir Guildgate, Hitchclaw Recluse, Skyline Scout, Hired Poisoner, Whisper Agent, Cosmotronic Wave, Mephitic Vapors, Thought Erasure, Swathcutter Giant, Demotion]",Thought Erasure,Dimir Guildgate
141,"[Dimir Guildgate, Whisper Agent, Hammer Dropper, Severed Strands, Ledev Guardian, Never Happened, Prey Upon, Leapfrog, Izzet Locket]",Whisper Agent,Dimir Guildgate
142,"[Disdainful Stroke, Izzet Locket, Severed Strands, Garrison Sergeant, Intrusive Packbeast, Crushing Canopy, Child of Night, Hellkite Whelp]",Disdainful Stroke,Disdainful Stroke
143,"[Boros Guildgate, Generous Stray, Boros Locket, Rosemane Centaur, Cosmotronic Wave, Devkarin Dissident, Street Riot]",Boros Guildgate,Boros Guildgate
144,"[Urban Utopia, Barrier of Bones, Severed Strands, Pack's Favor, Crushing Canopy, Undercity Necrolisk]",Severed Strands,Urban Utopia


In [226]:
df.iloc[180:225]

,options,predicted,taken
180,"[Vraska, Golgari Queen, Garrison Sergeant, Never Happened, Izzet Locket, Mephitic Vapors, Direct Current, Sumala Woodshaper, Luminous Bonds, Rhizome Lurcher, Ornery Goblin, Tenth District Guard, Hellkite Whelp, Justice Strike, Price of Fame, Boros Guildgate]","Vraska, Golgari Queen","Vraska, Golgari Queen"
181,"[Citywatch Sphinx, Golgari Locket, Moodmark Painter, Command the Storm, Izzet Locket, Unexplained Disappearance, Cosmotronic Wave, Collar the Culprit, Goblin Locksmith, Prey Upon, Pause for Reflection, Swarm Guildmage, Thought Erasure, Dimir Guildgate]",Citywatch Sphinx,Citywatch Sphinx
182,"[Whisper Agent, Collar the Culprit, Garrison Sergeant, Fresh-Faced Recruit, Torch Courier, Passwall Adept, Ornery Goblin, Skyline Scout, Urban Utopia, Electrostatic Field, District Guide, Might of the Masses, Golgari Guildgate]",Whisper Agent,Whisper Agent
183,"[Command the Storm, Vigorspore Wurm, Collar the Culprit, Cosmotronic Wave, Fresh-Faced Recruit, Hired Poisoner, Unexplained Disappearance, Maniacal Rage, Radical Idea, Wall of Mist, Goblin Cratermaker, Tajic, Legion's Edge]","Tajic, Legion's Edge",Command the Storm
184,"[Boros Challenger, Cosmotronic Wave, Sonic Assault, Mephitic Vapors, Collar the Culprit, Wary Okapi, Vicious Rumors, Boros Locket, Vigorspore Wurm, Wall of Mist, Inspiring Unicorn]",Boros Challenger,Boros Challenger
185,"[Luminous Bonds, Blade Instructor, Bartizan Bats, Maniacal Rage, Ledev Guardian, Sure Strike, Gravitic Punch, Generous Stray, Plaguecrafter, Electrostatic Field]",Luminous Bonds,Luminous Bonds
186,"[Sonic Assault, Undercity Uprising, Izzet Locket, Veiled Shade, Maniacal Rage, Ironshell Beetle, Hunted Witness, Molderhulk, Golgari Guildgate]",Ironshell Beetle,Sonic Assault
187,"[Inescapable Blaze, Vedalken Mesmerist, Sure Strike, Vicious Rumors, Take Heart, Ironshell Beetle, Devious Cover-Up, Grappling Sundew]",Inescapable Blaze,Inescapable Blaze
188,"[Hellkite Whelp, Garrison Sergeant, Never Happened, Izzet Locket, Ornery Goblin, Tenth District Guard, Justice Strike]",Justice Strike,Hellkite Whelp
189,"[Izzet Locket, Moodmark Painter, Unexplained Disappearance, Cosmotronic Wave, Goblin Locksmith, Pause for Reflection]",Unexplained Disappearance,Izzet Locket
